In [11]:
from pathlib import Path

# Detecta la raíz del repo (la que contiene 'src')
def find_project_root(marker="src", max_hops=7):
    p = Path.cwd()
    for _ in range(max_hops):
        if (p / marker).exists():
            return p
        p = p.parent
    raise RuntimeError(f"No se encontró la raíz del proyecto desde {Path.cwd()}")

PROJECT_ROOT = find_project_root()
DATA_DIR = PROJECT_ROOT / "data"  # <- carpeta 'data' de la raíz

print("[OK] Raíz del proyecto:", PROJECT_ROOT)
print("[OK] Carpeta data absoluta:", DATA_DIR)

[OK] Raíz del proyecto: c:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored
[OK] Carpeta data absoluta: c:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data


In [12]:
from datetime import date
from src.whoscored_viz.whoscored_fixtures import make_driver, scrape_range_finished

start_date = date(2025, 8, 1)
end_date   = date(2025, 8, 31)

# Usa la ruta absoluta hacia data/raw/fixtures en la raíz del repo
out_dir = str(DATA_DIR / "raw" / "fixtures")

driver = make_driver(use_uc=True, headless=True)
df_fixtures = scrape_range_finished(driver, start_date, end_date, out_dir)
driver.quit()

display(df_fixtures.head())

,match_date,start_time,home_name,away_name,match_id,match_centre_url,score_home,score_away,is_finished
0,2025-08-15,19:00,Girona,Rayo Vallecano,1913916,https://es.whoscored.com/Matches/1913916/Live,1,3,True
1,2025-08-15,21:30,Villarreal,Real Oviedo,1913892,https://es.whoscored.com/Matches/1913892/Live,2,0,True
2,2025-08-16,19:30,Mallorca,Barcelona,1913918,https://es.whoscored.com/Matches/1913918/Live,0,3,True
3,2025-08-16,21:30,Deportivo Alaves,Levante,1913913,https://es.whoscored.com/Matches/1913913/Live,2,1,True
4,2025-08-16,21:30,Valencia,Real Sociedad,1913889,https://es.whoscored.com/Matches/1913889/Live,1,1,True


In [1]:
from pathlib import Path
import pandas as pd

# Detectar raíz del proyecto (la carpeta que contiene 'src')
def find_project_root(marker="src", max_hops=7):
    p = Path.cwd()
    for _ in range(max_hops):
        if (p / marker).exists():
            return p
        p = p.parent
    raise RuntimeError(f"No se encontró la raíz del proyecto desde {Path.cwd()}")

PROJECT_ROOT = find_project_root()
DATA_DIR = PROJECT_ROOT / "data"

# Busca el último finished_matches.csv generado dentro de data/raw/fixtures/DataFixtures/**/
fixtures_root = DATA_DIR / "raw" / "fixtures" / "DataFixtures"
csv_paths = sorted(fixtures_root.rglob("finished_matches.csv"))
assert csv_paths, "No se encontró ningún finished_matches.csv. ¿Ejecutaste el Paso 2?"
fixtures_csv = csv_paths[-1]  # el más reciente

df_fix = pd.read_csv(fixtures_csv)
print("Fixtures CSV →", fixtures_csv)
df_fix.head(3)

Fixtures CSV → c:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\fixtures\DataFixtures\laliga\2025-2026\range_20250801_20250831\finished_matches.csv


,match_date,start_time,home_name,away_name,match_id,match_centre_url,score_home,score_away,is_finished
0,2025-08-15,19:00,Girona,Rayo Vallecano,1913916,https://es.whoscored.com/Matches/1913916/Live,1,3,True
1,2025-08-15,21:30,Villarreal,Real Oviedo,1913892,https://es.whoscored.com/Matches/1913892/Live,2,0,True
2,2025-08-16,19:30,Mallorca,Barcelona,1913918,https://es.whoscored.com/Matches/1913918/Live,0,3,True


In [4]:
# 3.2 que ya tenías: seleccionar un partido
row = df_fix.iloc[0]
match_url = str(row["match_centre_url"])

# Ejecuta por URL en vez de match-id
import sys, subprocess
from pathlib import Path

script_path = PROJECT_ROOT / "src" / "whoscored_viz" / "whoscored_matchcenter.py"

cmd = [
    sys.executable,
    str(script_path),
    "--url", match_url,  # <- aquí el cambio
]
print("Ejecutando:", " ".join(cmd))
res = subprocess.run(cmd, capture_output=True, text=True)
print("returncode:", res.returncode)
print("---- STDOUT ----\n", res.stdout[:2000])
print("---- STDERR ----\n", res.stderr[:2000])

Ejecutando: c:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\.venv\Scripts\python.exe c:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\src\whoscored_viz\whoscored_matchcenter.py --url https://es.whoscored.com/Matches/1913916/Live
returncode: 1
---- STDOUT ----
 
---- STDERR ----
 Traceback (most recent call last):
  File "c:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\src\whoscored_viz\whoscored_matchcenter.py", line 952, in <module>
    main()
  File "c:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\src\whoscored_viz\whoscored_matchcenter.py", line 940, in main
    res = process_one_match(
  File "c:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\src\whoscored_viz\whoscored_matchcenter.py", line 861, in process_one_match
    html = get_html_via_selenium(url, driver=driver, headless=headless)
  File "c:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\src\whoscored_viz\whoscored_matchcenter.py", line 80, in get_html_via_selenium
    WebDriverWait(driver, tim

In [5]:
# Prueba directa visible para inspección
from src.whoscored_viz.whoscored_matchcenter import get_html_via_selenium
html = get_html_via_selenium(match_url, headless=False, timeout=60)
print("HTML length:", len(html))

ModuleNotFoundError: No module named 'src'